Whats' released?

# Setup

In [1]:
from __future__ import print_function

In [2]:
import sys
import os
from IPython.display import clear_output
import RDF
import pandas

In [3]:
from curation_common import *

In [4]:
from htsworkflow.util.rdfhelp import fromTypedNode

In [5]:
%load_ext rdfmagic

The rdfmagic extension is already loaded. To reload it, use:
  %reload_ext rdfmagic


In [6]:
server = ENCODED('www.encodeproject.org')
server.load_netrc()

In [7]:
model = get_model(use_contexts=False)

# Load status from DCC

In [8]:
search = server.get_json('/search/?type=experiment&lab.title=Ali+Mortazavi%2C+UCI&award.rfa=ENCODE3')
len(search['@graph'])

166

In [9]:
for i, row in enumerate(search['@graph']):
    experiment_id = row['@id']
    #clear_output()
    print('loading', experiment_id, i, len(search['@graph']))
    experiment = server.get_jsonld(experiment_id)
    load_jsonld_into_model(model, experiment)

loading /experiments/ENCSR958UOC/ 0 166
loading /experiments/ENCSR894GAC/ 1 166
loading /experiments/ENCSR888ZAA/ 2 166
loading /experiments/ENCSR425ZMS/ 3 166
loading /experiments/ENCSR258UGT/ 4 166
loading /experiments/ENCSR326JQW/ 5 166
loading /experiments/ENCSR486QHW/ 6 166
loading /experiments/ENCSR101DVP/ 7 166
loading /experiments/ENCSR776XYQ/ 8 166
loading /experiments/ENCSR227VCK/ 9 166
loading /experiments/ENCSR630JWA/ 10 166
loading /experiments/ENCSR335LKF/ 11 166
loading /experiments/ENCSR396EAG/ 12 166
loading /experiments/ENCSR212RKJ/ 13 166
loading /experiments/ENCSR713JKD/ 14 166
loading /experiments/ENCSR752QGH/ 15 166
loading /experiments/ENCSR481QIP/ 16 166
loading /experiments/ENCSR571RHM/ 17 166
loading /experiments/ENCSR625ESQ/ 18 166
loading /experiments/ENCSR552WPH/ 19 166
loading /experiments/ENCSR415KUB/ 20 166
loading /experiments/ENCSR461SRF/ 21 166
loading /experiments/ENCSR070GAZ/ 22 166
loading /experiments/ENCSR008EQG/ 23 166
loading /experiments/ENCSR

# Current ENCODE3 Experiment Status

In [11]:
%%sparql -m model
select distinct ?o
{
    ?s a ?o .
}

o
https://www.encodeproject.org/profiles/Award.json#Award
https://www.encodeproject.org/profiles/Award.json#Item
biosample:Biosample
biosample:Item
https://www.encodeproject.org/profiles/Document.json#Document
https://www.encodeproject.org/profiles/Document.json#Item
experiment:Experiment
experiment:Dataset
experiment:Item
https://www.encodeproject.org/profiles/HumanDonor.json#HumanDonor


In [12]:
%%sparql -m model -c
select ?accession ?assay_name ?description ?status ?release
where {
  ?accession a experiment:Experiment ;
             rdf:description ?description ;
             experiment:assay_term_name ?assay_name ;
             experiment:status ?status .
  OPTIONAL { ?accession experiment:date_released ?release .}
}
order by ?status ?description

Found 166 rows.


accession,assay_name,description,status,release
experiments:ENCSR506JCY/,microRNA counts,Nanostring miRNA Array on embryonic 15.5 day mouse embryonic facial prominence,proposed,None
experiments:ENCSR853PWX/,microRNA counts,Nanostring miRNA Array on embryonic 15.5 day mouse kidney,proposed,None
experiments:ENCSR888ZAA/,microRNA counts,Nanostring miRNA Array on human heart left ventricle,proposed,None
experiments:ENCSR425ZMS/,microRNA counts,Nanostring miRNA Array on human heart left ventricle,proposed,None
experiments:ENCSR894GAC/,microRNA counts,Nanostring miRNA Array on human induced pluripotent stem cell,proposed,2016-05-03
experiments:ENCSR227VCK/,microRNA counts,Nanostring miRNA Array on human prostate,proposed,None
experiments:ENCSR630JWA/,microRNA counts,Nanostring miRNA Array on human prostate,proposed,None
experiments:ENCSR258UGT/,microRNA counts,Nanostring miRNA Array on human right cardiac atrium,proposed,None
experiments:ENCSR326JQW/,microRNA counts,Nanostring miRNA Array on human right cardiac atrium,proposed,None
experiments:ENCSR776XYQ/,microRNA counts,Nanostring miRNA Array on human right lobe of liver,proposed,None


# Current Encode3 Library status

In [12]:
%%sparql -m model
select ?p ?o
where {
    <https://www.encodeproject.org/organisms/mouse/> ?p ?o
}

p,o
rdf:type,https://www.encodeproject.org/profiles/Organism.json#Organism
rdf:type,https://www.encodeproject.org/profiles/Organism.json#Item
https://www.encodeproject.org/profiles/Organism.json#name,mouse
https://www.encodeproject.org/profiles/Organism.json#schema_version,2
https://www.encodeproject.org/profiles/Organism.json#scientific_name,Mus musculus
https://www.encodeproject.org/profiles/Organism.json#status,current
https://www.encodeproject.org/profiles/Organism.json#taxon_id,10090
https://www.encodeproject.org/profiles/Organism.json#uuid,3413218c-3d86-498b-a0a2-9a406638e786


In [13]:
%%sparql -m model -c -o biosamples
select ?biosample ?organism ?name
where {
    ?biosample a biosample:Biosample ;
               biosample:organism ?organism .
    ?organism a <https://www.encodeproject.org/profiles/organism.json#organism> ;
              <https://www.encodeproject.org/profiles/organism.json#name> ?name .
}

Found 0 rows.


In [14]:
biosample_map = {
    str(row['biosample']): str(row['name']) for row in biosamples
}

#for row in biosamples:
#    bio = str(row['biosample'])
#    name = str(row['name'])
#    biosample_map[bio] = name

In [ ]:
%%sparql -m model -c -o library_status
select ?experiment ?library ?biosample ?description ?alias ?status ?created ?date_released
where {
  ?experiment a experiment:Experiment ;
             rdf:description ?description ;
             experiment:status ?status ;
             experiment:replicates ?replicate .
   OPTIONAL { ?experiment experiment:date_released ?date_released .} 
   ?replicate replicate:library ?library .
   ?library library:date_created ?created ;
            library:biosample ?biosample .
   
   OPTIONAL { ?library library:aliases ?alias . }
}
order by ?created ?experiment

In [ ]:
records = []
for row in library_status:
    record = { k: str(row[k]) for k in row }
    #record['alias'] = record['alias'].replace('barbara-wold:', '')
    record['created'] = record['created'][:10]
    record['organism'] = biosample_map.get(record['biosample'], None)
    records.append(record)
librarydf = pandas.DataFrame(records, 
                             columns=['experiment', 'library', 'alias', 'organism', 'description', 'status', 'created'],
                             )

In [ ]:
librarydf.to_excel('/tmp/library.xlsx', index=False)

# Current Encode 3 File status

In [ ]:
%%sparql -m model -c -o file_status
select distinct ?experiment ?description ?status ?created
where {
  ?experiment a experiment:Experiment ;
             rdf:description ?description ;
             experiment:status ?status ;
             experiment:files ?file .
   ?file file:date_created ?created ;
         file:status ?status .
}
order by ?created ?status

In [ ]:
%%sparql -m model
select ?p ?o 
where {
  <https://www.encodeproject.org/libraries/ENCLB646JVH/> ?p ?o.
}

In [ ]:
%%sparql -m model
select ?p ?o 
where {
    <https://www.encodeproject.org/experiments/ENCSR950BNG/> ?p ?o
}

In [ ]:
%%sparql -m model
select ?p ?o
where {
    <https://www.encodeproject.org/replicates/a37a8cff-5bf7-47fc-b9a0-fa2aafebb9c3/> ?p ?o .
}

In [ ]:
%%sparql -m model
select ?p ?o 
where {
  <https://www.encodeproject.org/biosamples/ENCBS616ZXC/> ?p ?o
    }